In [ ]:
file_path = ""

In [ ]:
%matplotlib widget
from pathlib import Path
import matplotlib.pyplot as plt
import ncempy.io as nio
from ipywidgets import widgets
import h5py
import pickle

In [ ]:
print(f"Selected file: {file_path}")

In [ ]:
global emd
option = {}
opt = []

In [ ]:
num_pics = 0

In [ ]:
def get_file(file):
    global num_pics
    global opt
    emd = nio.emd.emdReader(file)
    num_pics = emd['data'].shape[0]
    
    for k in range(4):
        try:
            emd1 = (nio.emd.emdReader(file, dsetNum = k)['name'])
            option[emd1] = int(emd1[6:])
        except TypeError:
            pass
    for val in option.values():
        opt.append(val)


In [ ]:
%%capture 
# ^^hides the "group does not exist" output
get_file(file_path)

In [ ]:
image_number = widgets.IntSlider(
    value=1,
    min=1,
    max=num_pics,
    step=1,
    description='Image Number: '
)

size = widgets.Dropdown(
    options = opt,
    value = opt[0],
    description='Image Size'
)

In [ ]:
def change_pic (image_number, size):
    #read in selected emd data
    emd = nio.emd.emdReader(file_path, dsetNum = opt.index(size))
    
    #display image
    fig,ax = plt.subplots(1, 1)
    pic = plt.imshow(emd['data'][image_number-1])
    plt.show()   
    
    #print metadata
    print(f"Name: {emd['name']}")
    #print(f"Image Size: {emd['data'].shape[1]} {emd['pixelUnit'][1]}s by {emd['data'].shape[1]} {emd['pixelUnit'][1]}s")
    data_name = emd['name']
    with h5py.File(file_path, 'r') as f:
        pickledict = f[f"data/{emd['name']}/pickledict"][image_number-1] # take each images
        d = pickle.loads(pickledict, encoding='bytes')
        data = f[f"data/{emd['name']}/data"][image_number-1]
    
    metadata = {}
    metadata['Image Shape'] = d['_imageDataShape']
    metadata['Pixel Size'] = d['_calX'] # this assumes X and Y have the same pixel size 
    metadata['Brightness'] = d['_brightness'] # HAADF brightness 
    metadata['Contrast'] = d['_contrast'] # HAADF contrast
    metadata['Comment'] = d['_comment'] # image comment
    metadata['Camera Length'] = d['_cameralength'] # camera length label
    metadata['Camera Length Index'] = d['_cameralengthindex'] # camera length index
    metadata['Convergence Angle'] = d['_convergenceAngle'] # in radians
    metadata['Defocus'] = d['_defocus']
    metadata['Dwell Time'] = d['_dwell']
    metadata['Accelerating Voltage'] = d['_HTValue']
    metadata['Magnification'] = d['_mag']
    metadata['Stage Position'] = d['_position'] # X, Y, Z, Alpha, Gamma
    metadata['Scan Rotation'] = d['_rot'] # STEM scan rotation
    metadata['Field of View'] = [metadata['Pixel Size'] * metadata['Image Shape'][0], metadata['Pixel Size'] * metadata['Image Shape'][1]]
    
    #check comment note about leaving out if empty
    for k,v in metadata.items():
        if(k == 'Pixel Size'):
            print(f"{k}: {v*10**9:.2f}nm")
        elif(k == 'Comment' and v == "; "):
            pass
        elif(k == 'Camera Length'):
            print(f"{k}: {v*1000}mm")
        elif(k == 'Convergence Angle'):
            print(f"{k}: {v*1000:.2f}mrad")
        elif(k == 'Defocus'):
            print(f"{k}: {v*10**9:.2f}nm")
        elif(k == 'Dwell Time'):
            print(f"{k}: {v*10**6}μsec")
        elif(k == 'Accelerating Voltage'):
            print(f"{k}: {v/1000}kV")
        elif(k == "Image Shape"):
            print(f"{k}: {v[0]}pixels by {v[1]}pixels")
        elif(k == 'Stage Position'):
            print(f"{k}(X, Y, Z): {v[0]*10**6:.2f}μm, {v[1]*10**6:.2f}μm, {v[2]*10**6:.2f}μm \n{k}(Alpha, Gamma): {v[3]:.2f}degrees, {v[4]:.2f}degrees")
        elif(k == 'Scan Rotation'):
            print(f"{k}: {v:.1f}")
        elif(k == "Field of View"):
            print(f"{k}: {v[0]*10**9:.2f}nm by {v[1]*10**9:.2f}nm")
        else:
            print(f"{k}: {v}")

In [ ]:
widgets.interactive(change_pic, image_number = image_number, size=size)